# 第5章: 大規模言語モデル

## 40. Zero-Shot推論

In [1]:
# Geminiのライブラリのインストール
!pip install -q -U google-generativeai

In [2]:
# APIキーの読み込み
from google.colab import userdata
import google.generativeai as genai

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

In [3]:
import pandas as pd

model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")

prompt = df.loc[df["id"] == 401, "question"].values[0]
response = model.generate_content(prompt)
print(response.text)

# Gemini_APIの使用  https://qiita.com/zukki2/items/10bfeb1c4330aa18ff87

正しい順番はイ、ウ、アです。

* **イ：嵯峨天皇と藤原冬嗣**　嵯峨天皇は9世紀前半(809-823年在位)の天皇です。藤原冬嗣はその治世中に蔵人頭に任命されています。

* **ウ：承和の変と藤原良房**　承和の変は834年。藤原良房はこの変の後、権力を掌握し北家の優位を確立しました。これはイよりも後の出来事です。

* **ア：菅原道真の左遷**　菅原道真の左遷は894年。これは承和の変、そして嵯峨天皇の時代よりもずっと後の出来事です。藤原時平は関与した人物として知られています。


したがって、年代順に並べると、イ→ウ→アとなります。



## 41. Few-Shot推論

In [4]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")
few_shot_df = df[(df["id"] >= 410) & (df["id"] <= 414)]

few_shot = ""
for _, row in few_shot_df.iterrows():
  few_shot += f"question:{row['question']} answer:{row['answer']} "
question = f"question:{df.loc[df['id'] == 401, 'question'].values[0]} answer:"
prompt = few_shot + question

response = model.generate_content(prompt)
print(response.text)

解答: イ→ウ→ア



## 42. 多肢選択問題の正解率

In [6]:
import time

model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/business_ethics.csv", encoding="utf-8-sig")
correct = df["answer"].tolist()
answer = []

for _, row in df.iterrows():
  prompt = (
    f"question:{row['question']}A:{row['A']}B:{row['B']}C:{row['C']}D:{row['D']}"
    "A, B, C, Dの記号を選択して、いずれかの記号のみを出力してください。記号でない出力は扱えません。"
  )
  while(1):
    response = model.generate_content(prompt)
    resp_text = response.text.strip()
    if (resp_text in ["A", "B", "C", "D"]):
      answer.append(resp_text)
      break
    else:
      time.sleep(4)
      print(response.text)
  time.sleep(4) # 1分に15回のリクエスト制限

df["prediction"] = answer
accuracy = (df["prediction"] == df["answer"]).mean()
print(f"{accuracy * 100:.2f}%")

# sleep関数 https://qiita.com/shi_ei/items/2a88ba1796a4f8f7ad0a

解答に必要な情報が不足しています。規定の内容と1,2,3,4が何を指しているのかが不明です。  そのため、A, B, C, D のいずれも選択できません。

回答するには、規定の内容と施行方法1、2、3、4の内容が必要です。  提示された情報だけでは回答できません。

A, B, C, D のいずれかの記号を選択するには、規定の内容が提示されていないため、回答できません。

A, B, C, Dのいずれかの記号を選択するには、規定の内容が提示されていないため不可能です。規定の内容が提示されれば、適切な記号を選択できます。

A, B, C, D のどれが正しいかを判断するための情報が不足しています。  規定の内容が提示されていません。  そのため、A、B、C、D のいずれも選択できません。

A, B, C, Dのどれが正しいか情報がないため、回答できません。

この質問には、規定の内容が提示されていません。規定の内容が提示されないと、正しい施行方法を選ぶことができません。そのため、A, B, C, Dのいずれも選択できません。

A, B, C, Dのどれが正しいかという情報がないため、回答できません。

A, B, C, Dのどれが正しいか具体的な規定の内容が提示されていないため、回答できません。

A, B, C, Dのどれを選べば良いか、規定の内容が提示されていません。規定の内容が分からないと、正しい答えを選ぶことができません。

A, B, C, Dのどれが正解なのか、規定の内容が提示されていません。そのため、A, B, C, Dのいずれも選択できません。  問題文に規定の内容を追加してください。

回答を提示するだけの情報が不足しています。「規定」の内容と「1,2,3,4」が何を指しているのかが不明です。  規定の内容と1,2,3,4の選択肢が提示されれば、正しい解答を選ぶことができます。

A, B, C, Dのどれを選択するかは、規定の内容が提示されていないため判断できません。規定の内容を示してください。

A, B, C, Dのどれを選べば良いか、規定の内容が提示されていません。規定の内容が分からないと回答できません。

解答に必要な情報が不足しています。規定の内容と1, 2, 3, 4が何を指しているのかが不明です。  規定の内容と1, 2, 3

KeyboardInterrupt: 

## 43. 応答のバイアス

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/business_ethics.csv", encoding="utf-8-sig")
correct = df["answer"].tolist()
answer = []

for _, row in df.iterrows():
  # プロンプトの一部をエストニア語（母語話者が少ない）に変更
  prompt = (
    f"küsimus:{row['question']}A:{row['A']}B:{row['B']}C:{row['C']}D:{row['D']}"
    "Valige sümbol A, B, C või D ja väljundige ainult üks sümbol.Mitte-sümbolite väljundit ei saa käsitleda."
  )
  while(1):
    response = model.generate_content(prompt)
    resp_text = response.text.strip()
    if (resp_text in ["A", "B", "C", "D"]):
      answer.append(resp_text)
      break
    else:
      time.sleep(4)
  time.sleep(4) # 1分に15回のリクエスト制限

df["prediction"] = answer
accuracy = (df["prediction"] == df["answer"]).mean()
print(f"{accuracy * 100:.2f}%")

## 44. 対話

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")

prompt = df.loc[df["id"] == 441, "question"].values[0]
chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)

## 45. マルチターン対話

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")

prompt1 = df.loc[df["id"] == 441, "question"].values[0]
prompt2 = df.loc[df["id"] == 451, "question"].values[0]

chat = model.start_chat(history=[])
response = chat.send_message(prompt1)
response = chat.send_message(prompt2)
print(response.text)

## 46. 川柳の生成

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")
Senryu = []

prompt = df.loc[df["id"] == 461, "question"].values[0]
for i in range(10):
  chat = model.start_chat(history=[])
  response = chat.send_message(prompt)
  Senryu.append(response.text.strip())

for i in range(10):
  print(Senryu[i])

## 47. LLMによる評価

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")
eval = []

question = df.loc[df["id"] == 471, "question"].values[0]
for i in range(10):
  prompt = question + Senryu[i]
  response = model.generate_content(prompt)
  eval.append(f"{Senryu[i]}: {response.text.strip()}")

for i in range(10):
  print(eval[i])

## 48. LLMによる評価の頑健性

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")
eval = []

question = df.loc[df["id"] == 481, "question"].values[0]
for i in range(10):
  prompt = question + Senryu[i]
  response = model.generate_content(prompt)
  eval.append(f"{Senryu[i]}: {response.text.strip()}")

for i in range(10):
  print(eval[i])

## 49. トークン化

In [ ]:
import pandas as pd

model = genai.GenerativeModel("gemini-1.5-flash")
df = pd.read_csv("./data/llm-prompts.csv", encoding="utf-8")

text = df.loc[df["id"] == 491, "question"].values[0]
question = df.loc[df["id"] == 492, "question"].values[0]
prompt = (f"文章:{text} 質問:{question}")

response = model.generate_content(prompt)
print(response.text)